In [1]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot') 
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler
from nitime import algorithms as alg
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 'pineline 13.csv', 'pipe line 10', 'pipe line 10.csv', 'pipeline 13.pickle', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'spindle training.p', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final

In [ ]:
resolution = 1;spindle_training_data=[];non_spindle_training_data=[];spindle_labels=[];non_spindle_labels=[]

# where two intervals overlap
def getOverlap(a,b):
    return max(0,min(a[1],b[1]) - max(a[0],b[0]))

result = {}
for num, EDFfileName in enumerate(EDFfiles):
    if EDFfileName == 'suj13_l2nap_day2 edited.edf' or EDFfileName =='suj13_l2nap_day2 edited1.edf':
        pass
    else:
        file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=num)
        # channel list for artifact(eyemovement or chewing) removal - ROC and/or LOC
        channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
        raw = eegPinelineDesign.load_data(file_to_read,channelList,None, None)# No pass
        annotation_to_read = [x for x in Annotationfiles if fileName in x]
        file = pd.read_csv(annotation_to_read[0])
        labelFind = re.compile('spindle',eegPinelineDesign.re.IGNORECASE)
        spindles=[]# take existed annotations
        for row in file.iterrows():
            currentEvent = row[1][-1]
            if labelFind.search(currentEvent):
                spindles.append(row[1][0])# time of marker
        print('finish spindle annotation')
        # threshold test to detect possible spindle segments
        channelList=['F3','F4','C3','C4','O1','O2']
        windowsize = 5;ratio_index={};muscle_index={};sample_ind={}
        print('start threshold')
        for ii,names in enumerate(channelList):
            #initialize
            endPoint=windowsize;startPoint=endPoint-windowsize
            distance_to_end = raw.last_samp/1000 - endPoint
            ratio_index[names] =[];muscle_index[names]=[];sample_ind[names]=[];

            # start the while loop
            print('start the while loop')
            while distance_to_end > windowsize:
                start, stop = raw.time_as_index([startPoint,endPoint])
                tempSegment, time = raw[ii,start:stop]



                filter_alpha = mne.filter.band_pass_filter(tempSegment[0,:],1000,8,12.5) #band pass alpha
                RMS_alpha = np.sqrt(sum(filter_alpha**2)/len(filter_alpha))

                filter_spindle = mne.filter.band_pass_filter(tempSegment[0,:],1000,11.5,16.5)# band pass 11.5 - 16
                RMS_spindle = np.sqrt(sum(filter_spindle**2)/len(filter_spindle))

                filter_muscle = mne.filter.band_pass_filter(tempSegment[0,:],1000,30,40.5) # band pass 30 - 40, muscle movement
                RMS_muscle = np.sqrt(sum(filter_muscle**2)/len(filter_muscle))

                point=[0,0]
                if RMS_alpha/RMS_spindle < 1.2:# very subjective value
                    point[0] += 2
                    ratio_index[names].append(2)
                else:
                    point[0] += -1
                    ratio_index[names].append(-1)

                if RMS_muscle > 5*10e-4:
                    point[1] += -1
                    muscle_index[names].append(-1)
                else:
                    point[1] += 1
                    muscle_index[names].append(1)

                sample_ind[names].append(sum(point))

                endPoint += resolution
                startPoint=endPoint - windowsize
                distance_to_end = raw.last_samp/1000 - endPoint
            sample_ind[names].append(0)
        general_result = pd.DataFrame(sample_ind)
        print('finish the while loop')
        mark_pairs={};indicator={};print('start mark pairs')
        for ii,names in enumerate(channelList):
            mark_pairs[names]=[];indicator[names]=[]
            for pos,items in enumerate(general_result[names]):
                if pos == len(general_result[names])-1:
                    pass
                else:

                    indicator[names].append(general_result[names][pos+1]-general_result[names][pos])

                    if indicator[names][pos] == -3:
                        endPoint = pos*resolution
                        initial_position=pos;position_idx=pos
                        try:
                            while indicator[names][initial_position] != 3:
                                position_idx -= 1 # backward search
                                initial_position = position_idx
                            startPoint = position_idx * resolution

                            if endPoint - startPoint >= 1:
                                mark_pairs[names].append((startPoint,endPoint))

                        except:
                            pass

        # start to put in spindle event and non events
        print('start putting in spindle instances')
        cnt = 0
        for instance in spindles:
            tempData=[]
            for ii, names in enumerate(channelList):
                startPoint=instance-2.5;endPoint=instance+2.5
                start,stop=raw.time_as_index([startPoint,endPoint])
                tempSegment,timeSpan = raw[ii,start:stop]
                centerxval=timeSpan[np.argmax(abs(tempSegment[0,:]))]
                startPoint=centerxval-1.5;endPoint=centerxval+1.5
                start,stop=raw.time_as_index([startPoint,endPoint])
                tempSegment,time = raw[ii,start:stop]
                tempData = np.append(tempData,tempSegment)
            spindle_training_data.append(tempData)
            spindle_labels.append(1)
            cnt +=1
        print('start puting in non spindle instances')
        used_time_windows=[]
        for ii in range(cnt):
            Condition_operator=[False,False,False,False,False,False,False,False]

            while sum(np.array(Condition_operator).astype(int)) < 7:
                instance = np.random.choice(np.arange(raw.last_samp/1000),1)
                startPoint = instance - 1.5; endPoint = instance + 1.5

                for idx, names in enumerate(channelList):
                    Condition_operator[idx]=any(getOverlap([instance -1,instance + 1],[lower,upper]) !=0 for (lower,upper) in mark_pairs[names])

                Condition_operator[6]=all(getOverlap([startPoint,endPoint],[instance-1.5,instance+1.5])==0 for instance in spindles)
                Condition_operator[7]=all(getOverlap([startPoint,endPoint],[lower,upper])!=0 for (lower,upper) in used_time_windows)
                print(Condition_operator)
            used_time_windows.append([startPoint,endPoint])
            tempData=[]
            for idxx,names in enumerate(channelList):
                startPoint=instance-1.5;endPoint=instance+1.5
                start,stop=raw.time_as_index([startPoint,endPoint])
                tempSegment,time = raw[idxx,start:stop]
                tempData = np.append(tempData,tempSegment)
            non_spindle_training_data.append(tempData)
            non_spindle_labels.append(0)
            print(ii)
        print(EDFfileName,len(spindle_training_data),len(non_spindle_training_data))
        result[EDFfileName]={'spindle training':spindle_training_data,'spindle label':spindle_labels,
                            'non spindle training':non_spindle_training_data,'non spindle label':non_spindle_labels}

import pickle
with open('spindle training.p','rb') as handle:
    pickle.dump(result,handle)

In [2]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot') 
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler
from nitime import algorithms as alg
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 'pineline 13.csv', 'pipe line 10', 'pipe line 10.csv', 'pipeline 13.pickle', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'spindle training.p', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final

C:\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


In [3]:
import pickle
with open('spindle training.p','rb') as handle:
    result=pickle.load(handle)

In [4]:
result.keys()

dict_keys(['suj8_d1final.edf', 'suj5_d1final.edf', 's5d2_final.edf', 's6n2_final.edf', 'suj9_d2final.edf', 'suj10_d1final.edf', 'suj9_d1final.edf', 'suj6_d1final.edf', 'suj8_d2final.edf', 'suj10_d2final.edf'])

In [5]:
for items in result.items():
    print(items[0],len(items[1]['spindle training']))

suj8_d1final.edf 1363
suj5_d1final.edf 1363
s5d2_final.edf 1363
s6n2_final.edf 1363
suj9_d2final.edf 1363
suj10_d1final.edf 1363
suj9_d1final.edf 1363
suj6_d1final.edf 1363
suj8_d2final.edf 1363
suj10_d2final.edf 1363


In [6]:
spindle_training_data = result['suj9_d1final.edf']['spindle training'];print(len(spindle_training_data))
non_spindle_training_data = result['suj9_d1final.edf']['non spindle training'];print(len(non_spindle_training_data))
spindle_label=result['suj9_d1final.edf']['spindle label'];print(len(spindle_label))
non_spindle_label=result['suj9_d1final.edf']['non spindle label'];print(len(non_spindle_label))

1363
1363
1363
1363


In [7]:
for ii in range(len(spindle_training_data)):
    spindle_training_data.append(non_spindle_training_data[ii])
    spindle_label.append(non_spindle_label[ii])
    print(len(spindle_training_data),len(spindle_label))

1364 1364
1365 1365
1366 1366
1367 1367
1368 1368
1369 1369
1370 1370
1371 1371
1372 1372
1373 1373
1374 1374
1375 1375
1376 1376
1377 1377
1378 1378
1379 1379
1380 1380
1381 1381
1382 1382
1383 1383
1384 1384
1385 1385
1386 1386
1387 1387
1388 1388
1389 1389
1390 1390
1391 1391
1392 1392
1393 1393
1394 1394
1395 1395
1396 1396
1397 1397
1398 1398
1399 1399
1400 1400
1401 1401
1402 1402
1403 1403
1404 1404
1405 1405
1406 1406
1407 1407
1408 1408
1409 1409
1410 1410
1411 1411
1412 1412
1413 1413
1414 1414
1415 1415
1416 1416
1417 1417
1418 1418
1419 1419
1420 1420
1421 1421
1422 1422
1423 1423
1424 1424
1425 1425
1426 1426
1427 1427
1428 1428
1429 1429
1430 1430
1431 1431
1432 1432
1433 1433
1434 1434
1435 1435
1436 1436
1437 1437
1438 1438
1439 1439
1440 1440
1441 1441
1442 1442
1443 1443
1444 1444
1445 1445
1446 1446
1447 1447
1448 1448
1449 1449
1450 1450
1451 1451
1452 1452
1453 1453
1454 1454
1455 1455
1456 1456
1457 1457
1458 1458
1459 1459
1460 1460
1461 1461
1462 1462
1463 1463


In [8]:
training_data_temp = spindle_training_data
training_label_temp = spindle_label

In [9]:
training_data=[];training_label=[]
for items in zip(training_data_temp,training_label_temp):
    if len(items[0]) == 18000:
        training_data.append(items[0])
        training_label.append(items[1])
print(np.array(training_data).shape,np.array(training_label).shape)

(2720, 18000) (2720,)


In [10]:
print(training_label[:5])
np.random.shuffle(training_data)
np.random.shuffle(training_label)
print(training_label[:5])

[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(training_data, training_label, test_size=0.20, random_state=42)

In [12]:
#clf = OneVsRestClassifier(SVC(kernel='linear',tol=1e-6,max_iter=5e5))
clf = OneVsRestClassifier(LogisticRegression(penalty='l2',C=5,tol=10e-9,fit_intercept=True,solver='liblinear',
                                             max_iter=5e8,multi_class='ovr',n_jobs=-1))



clf.fit(X_train,y_train)
print(classification_report(clf.predict(X_test),y_test))
print(clf.score(X_train,y_train))

             precision    recall  f1-score   support

          0       0.32      0.55      0.40       176
          1       0.67      0.45      0.54       368

avg / total       0.56      0.48      0.49       544

0.543658088235


In [13]:
classification = make_pipeline(StandardScaler(),clf)
n_samples=np.array(training_data).shape[0]
cv = ShuffleSplit(n_samples, n_iter=3,test_size=0.3, random_state=0)
scores = cross_val_score(classification, np.array(training_data),np.array(training_label),cv=cv, scoring='accuracy')

print(scores)

[ 0.50612745  0.51960784  0.46078431]


In [14]:
clf = LDA(solver='svd', shrinkage=None, priors=None, n_components=100, store_covariance=False, tol=10e-8)
clf.fit(X_train,y_train)
print(classification_report(clf.predict(X_test),y_test))
print(clf.score(X_train,y_train))

             precision    recall  f1-score   support

          0       0.50      0.57      0.53       265
          1       0.53      0.47      0.50       279

avg / total       0.52      0.52      0.52       544

0.929227941176


C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [15]:
classification = make_pipeline(StandardScaler(),clf)
n_samples=np.array(training_data).shape[0]
cv = ShuffleSplit(n_samples, n_iter=3,test_size=0.3, random_state=0)
scores = cross_val_score(classification, np.array(training_data),np.array(training_label),cv=cv, scoring='accuracy')

print(scores)

C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[ 0.48897059  0.52573529  0.49264706]


C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
